# PyTorch Training Bootcamp
## Toradex + Inova.USP

18/11/2023

Elaborado por: Bruno Mello (bruno.mello@toradex.com)

## Preparação

## Configuração do ambiente

#### Download do Dataset

Neste exemplo estamos usando o dataset [Fruits and Vegetables Image Recognition Dataset](https://www.kaggle.com/datasets/kritikseth/fruit-and-vegetable-image-recognition), disponível sob a licensa [CC0: Public Domain](https://creativecommons.org/publicdomain/zero/1.0/).

Por estar disponível sob uma licensa de domínio público, podemos fazer basicamente o que quisermos com esse dataset, inclusive redistribuir ele.


O dataset a ser usado na avaliação está disponível em:

- Treinamento: https://docs.toradex.com/private/114105-recyclables_train.zip​
- Validação: https://docs.toradex.com/private/114106-recyclables_validation.zip


In [ ]:
!mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


In [ ]:
!# Baixar e descompactar o dataset
!wget -q https://docs.toradex.com/private/114105-recyclables_train.zip​

In [ ]:
!unzip -q '/content/114105-recyclables_train.zip​' -d dataset/

replace dataset/train_crops/canister/canister/prepared_data_all_MGS-27-Oct_10-01-13_01.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Baixar e descompactar o dataset
!wget -q https://docs.toradex.com/private/114106-recyclables_validation.zip

In [ ]:
!unzip -q '/content/114106-recyclables_validation.zip' -d dataset/

In [ ]:
data_path = "dataset"

### Organização de arquivos
O menu lateral esquerdo pode ser usado para navegar sobre os arquivos.

Por exemplo: /content/dataset/train/apple/Image_1.jpg

In [ ]:
from IPython.display import Image
Image("/content/dataset/validation_crops/bottle/bottle-blue/Monitoring_photo_2_test_25-Mar_11-13-25_01.jpg", width=256)

#### Instalação e Carregamento de Dependências

In [ ]:
# Carregar a extensão do TensorBoard para o Google Colab
%load_ext tensorboard

In [ ]:
# Instalar Pytorch, Torchvision, Tensorboard e utilidades parar ver o progresso do treinamento
!pip install -q torch torchvision torcheval tensorboard matplotlib tqdm tensorflow ipywidgets seaborn

## Código Fonte

### Importação de dependências

In [ ]:
# Módulos, classes e funções que são úteis para inferência e treinamento
import torch, torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_image
from torcheval.metrics import MulticlassF1Score, MulticlassRecall, MulticlassPrecision

# Suporte a TensorBoard no PyTorch
from torch.utils.tensorboard import SummaryWriter

# Utiliades do sistema
from datetime import datetime
import time
import os

# O Tqdm é utilizado para criar barras de progresso
from tqdm.notebook import tqdm


print(torch.__version__)

### Dataset Customizado

Implementamos um dataset customizado `CustomDataset`, que herda a classe `torch.utils.data.Dataset`. Essa nova classe foi feita para carregar um dataset organizado exatamente como o nosso está. Se o dataset estiver organizado de alguma outra forma, essa classe não será adequada para isso.

Essa é a forma recomendada pela documentação do Pytorch de lidar com datasets, um pouco do background do porquê pode ser visto na página [Datasets & Dataloaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) da documentação.

Também existem muitos [datasets disponíveis diretamente no PyTorch](https://pytorch.org/vision/stable/datasets.html), porém deve-se tomar cuidado com a licensa de uso para esses datasets, nem sempre uso comercial é permitido.

In [ ]:
class CustomDataset(Dataset):

    # Construtor
    def __init__(self, images_dir, preprocess_function):

        """
        Args:
            images_dir (string): Directory with all the image folders
            preprocess_function (callable): Transform to be applied on a sample
        """

        self.images_dir = images_dir
        self.transform = preprocess_function

        # Ordenamos as classes para termos sempre a mesma ordem
        # Aqui ficam armazenados os nomes de cada classe
        self.classes = []
        pasta = os.listdir(self.images_dir)
        self.super_classes = sorted(os.listdir(self.images_dir))

        # Caminho para cada imagem
        self.image_paths = []
        self.image_classes = []

        contador = -1
        # Procura imagens para todas as classes, em suas respectivas pastas
        for i in range(0, len(self.super_classes)):
          sample_class = self.super_classes[i]
          class_dir = os.path.join(self.images_dir, sample_class)
          class_images = sorted(os.listdir(class_dir))

          for sub_pastas in class_images:
            caminho = os.path.join(class_dir, sub_pastas)
            nova_lista = os.listdir(caminho)
            contador = contador + 1
            self.classes.append(sub_pastas)

            for image in nova_lista:
              if(not (image.endswith(".jpg") or image.endswith(".JPG") or image.endswith(".png") or image.endswith(".PNG"))):
                  continue
              image_path = os.path.join(class_dir, sub_pastas, image)
              self.image_paths.append(image_path)
              self.image_classes.append(contador)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Checagem por erros na leitura da imagem
        # Isso gera uma nova excessão para que o dataset seja corrigido, não queremos imagens com problemas no dataset
        try:
            sample = read_image(self.image_paths[idx], torchvision.io.ImageReadMode.RGB)
        except:
            print(f"Problem loading image {self.image_paths[idx]}")
            raise Exception()

        # Preprocessamento da imagem
        sample = self.transform(sample)

        # Retorno do par (imagem, classe)
        return sample, self.image_classes[idx]


### Pré-processamento

Na etapa de pré-processamento, deve-se formatar os dados ao formato esperado pelo modelo utilizado.

Aqui, a imagem é primeiro redimensionada para a resolução de `inference_size`x`inference_size`, depois convertida para ponto flutuante (float) e depois normalizada. Os valores de normalização `NormalizationMean` e `NormalizationStd` são dados pelo modelo utilizado. No caso ResNet, disponível no [PyTorch Hub](https://pytorch.org/hub/pytorch_vision_resnet/). O uso dos `transforms` do PyTorch facilita o preprocessamento de dados.

O `batch_size` representa o número de imagens que será processada de uma vez. Isso é útil para atingir melhor paralelismo na inferência e no treinamento do modelo utilizado, mas tem um impacto no tempo de inferência e no uso de memória RAM.

In [ ]:
batch_size = 4
inference_size = 160
NormalizationMean = [0.485, 0.456, 0.406]
NormalizationStd = [0.229, 0.224, 0.225]

preprocess_image = transforms.Compose([
    transforms.Resize((inference_size, inference_size), antialias=True),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(mean=NormalizationMean, std=NormalizationStd),
])

### Instanciamento dos datasets

Inicialmente definimos o caminho em que os datasets estão, relativos ao `data_path` que definimos quando baixamos e descompactamos o dataset.

In [ ]:
train_data_path = os.path.join(data_path, "train_crops")
validation_data_path = os.path.join(data_path, "validation_crops")

In [ ]:
test_data_path = os.path.join(data_path, "test_crops") #ARRUMAR AQUI

Agora, inicializamos as classes `CustomDataset` e `DataLoader` para os datasets que temos:

- Treinamento: Grande dataset, será usado para ajustar os pesos do modelo
- Validação: Menor dataset, será usado para avaliar o modelo treinado e escolher o checkpoint com melhor performance
- Teste: Menor dataset, será usado para medir a performance do checkpoint escolhido durante o treinamento

É importante ressaltar que os datasets não devem ter imagens em comum entre si.

In [ ]:
train_dataset = CustomDataset(train_data_path, preprocess_image)
validation_dataset = CustomDataset(validation_data_path, preprocess_image)
# test_dataset = CustomDataset(test_data_path, preprocess_image)

print(train_dataset)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
#test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

print( "Train dataset information:")
print(f" |-> Path:          {train_data_path}")
print(f" |-> Length:        {len(train_dataset)}")
print(f" '-> Loader length: {len(train_dataloader)}")

print( "Validation dataset information:")
print(f" |-> Path:          {validation_data_path}")
print(f" |-> Length:        {len(validation_dataset)}")
print(f" '-> Loader length: {len(validation_dataloader)}")

#

### Carregar o modelo

Existem diferentes tamanhos para o modelo resnet, quanto maior o modelo, mais preciso ele consegue ficar, mas ao mesmo tempo, maior é o tempo de execução.

A resolução da imagem usada na inferência também tem influência no tempo de execução, em um sistema embarcado muitas vezes é necessário avaliar a melhor combinação de resolução e tamanho de modelo para ter uma performance aceitável, tanto em termos de qualidade da saída quanto em termos de uso de recursos computacionais limitados.

In [ ]:
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights=torchvision.models.ResNet18_Weights.DEFAULT)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', weights=torchvision.models.ResNet34_Weights.DEFAULT)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights=torchvision.models.ResNet50_Weights.DEFAULT)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', weights=torchvision.models.ResNet101_Weights.DEFAULT)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', weights=torchvision.models.ResNet152_Weights.DEFAULT)
model.eval();

### TensorBoard

O TensorBoard é utilizado para visualizar estatísticas de treinamento e acompanhar o progresso em tempo real.

No Google Colab, ele deve ser inicializado antes de se iniciar o treinamento.

In [ ]:
%tensorboard --logdir runs

### Função para treinar o modelo por uma época

Treinar o modelo é, de forma simplificada, otimizar os parâmetros da grande equação que é o modelo para que os resultados sejam mais próximos do esperado. Isso é feito usando uma heurística de perda (loss), que é minimizada.

Uma época neste caso é definida como uma iteração do treinamento para cada amostra do dataset de treinamento.

In [ ]:
def train_one_epoch(model, dataloader, optimizer, loss_function, epoch_index, device, tensorboard_writer):
    running_loss = 0.0
    last_loss = 0.0


    model.to(device)

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    print(f"Training Epoch {epoch_index}:")

    progress_bar = tqdm(total=len(dataloader))
    for i, data in enumerate(dataloader):

        # Every data instance is an input + label pair
        inputs, labels = data


        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero gradients for every batch
        optimizer.zero_grad()

        # Run Inference on the training data
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_function(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 10 == 9:
            last_loss = running_loss / 10 # loss per batch
            tb_x = (epoch_index*len(train_dataloader)) + i
            tensorboard_writer.add_scalar('Loss/train', last_loss, tb_x)
            tensorboard_writer.flush()
            running_loss = 0.0

        del inputs, labels

        progress_bar.update(1)
    return last_loss

In [ ]:

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
tensorboard_writer = SummaryWriter(f'runs/model_{timestamp}')

EPOCHS = 6

best_validation_loss = -1

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.008, momentum=0.9)

device = 'cpu'
if(torch.cuda.is_available()):
    device = 'cuda'

for epoch in range(EPOCHS):
    print(f'EPOCH {epoch}:')

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(model, train_dataloader, optimizer, loss_function, epoch, device, tensorboard_writer)

    running_validation_loss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    model.to(device)

    precision_metric = MulticlassPrecision(average='weighted', num_classes=len(validation_dataset.classes))
    recall_metric = MulticlassRecall(average='weighted', num_classes=len(validation_dataset.classes))
    f1score_metric = MulticlassF1Score(average='weighted', num_classes=len(validation_dataset.classes))

    # Disable gradient computation and reduce memory consumption.
    print(f"Running Validation for Epoch {epoch}")
    with torch.no_grad():
        progress_bar = tqdm(total=len(validation_dataloader))
        for validation_data in validation_dataloader:
            validation_inputs, validation_labels = validation_data
            validation_inputs = validation_inputs.to(device)
            validation_labels = validation_labels.to(device)
            validation_outputs = model(validation_inputs)
            running_validation_loss += loss_function(validation_outputs, validation_labels)

            output_labels = []

            for j in range(0, validation_outputs.size()[0]):
                probabilities = torch.nn.functional.softmax(validation_outputs[j], dim=0)
                prob, det_class = torch.topk(probabilities, 1)
                # If the detected class is outside of bounds, give a wrong result within bounds
                if(det_class >= len(validation_dataset.classes)):
                    det_class = validation_labels[j].item()-1
                    if(det_class < 0):
                        det_class = validation_labels[j].item()+1
                output_labels.append(det_class)

            output_labels = torch.as_tensor(output_labels)
            output_labels.to(device)

            precision_metric.update(output_labels, validation_labels)
            recall_metric.update(output_labels, validation_labels)
            f1score_metric.update(output_labels, validation_labels)

            del validation_inputs, validation_labels, output_labels

            progress_bar.update(1)

    precision = precision_metric.compute().item()
    recall = recall_metric.compute().item()
    f1score = f1score_metric.compute().item()

    average_validation_loss = running_validation_loss / len(validation_dataloader)

    # Log the running loss averaged per batch
    # for both training and validation
    tensorboard_writer.add_scalars('Training vs. Validation Loss',
                                  { 'Training': avg_loss,
                                    'Validation': average_validation_loss},
                                    epoch)

    tensorboard_writer.add_scalars('Validation Metrics',
                                  { 'Precision': precision,
                                    'Recall': recall,
                                    'F1 Score': f1score},
                                    epoch)
    tensorboard_writer.flush()

    tensorboard_writer.add_scalars('Learning Rate',
                                  {'lr': optimizer.state_dict()['param_groups'][0]['lr']},
                                    epoch)
    tensorboard_writer.flush()

    # Track best performance, and save the model's state
    if((average_validation_loss < best_validation_loss) or (best_validation_loss == -1)):
        best_validation_loss = average_validation_loss
        torch.save(model.state_dict(), f'model_{timestamp}_best')

    torch.save(model.state_dict(), f'model_{timestamp}_{epoch}')


### Teste do modelo

A Função abaixo serve para testar o modelo, calculando uma matriz de confusão, precisão, recall e F1 Score.
A Função também mede o tempo de execução, mas isso não é reproduzível o suficiente no Google Colab.

Notebook do Google Colab com ajustes esperados: https://colab.research.google.com/drive/1c5JcE1FtDuoL8bnZC7ST9vsveBgDqehS?usp=sharing


In [ ]:
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from torcheval.metrics import MulticlassConfusionMatrix

def test_model(model, device, test_dataset, batch_size=1, run_tests=True, time_test_iterations=1000, show_confusion_matrix=True, test_time=True):

    if(run_tests and show_confusion_matrix):
        confusion_matrix_metric = MulticlassConfusionMatrix(num_classes=len(test_dataset.classes))

    if(run_tests):
        precision_metric = MulticlassPrecision(average='weighted', num_classes=len(test_dataset.classes))
        recall_metric = MulticlassRecall(average='weighted', num_classes=len(test_dataset.classes))
        f1score_metric = MulticlassF1Score(average='weighted', num_classes=len(test_dataset.classes))

    model.eval()
    model.to(device)

    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

    if(run_tests):
        progress_bar = tqdm(total=len(test_dataloader))
        for test_data in test_dataloader:
            test_inputs, test_labels = test_data
            test_inputs = test_inputs.to(device)
            test_labels = test_labels.to(device)
            test_outputs = model(test_inputs)
            output_labels = []

            for j in range(0, test_outputs.size()[0]):
                probabilities = torch.nn.functional.softmax(test_outputs[j], dim=0)
                prob, det_class = torch.topk(probabilities, 1)
                # If the detected class is outside of bounds, give a wrong result within bounds
                if(det_class >= len(test_dataset.classes)):
                    det_class = test_labels[j].item()-1
                    if(det_class < 0):
                        det_class = test_labels[j].item()+1
                output_labels.append(det_class)

            output_labels = torch.as_tensor(output_labels)
            output_labels.to(device)

            confusion_matrix_metric.update(output_labels, test_labels)
            precision_metric.update(output_labels, test_labels)
            recall_metric.update(output_labels, test_labels)
            f1score_metric.update(output_labels, test_labels)

            del test_inputs, test_labels

            progress_bar.update(1)

        precision = precision_metric.compute().item()
        recall = recall_metric.compute().item()
        f1score = f1score_metric.compute().item()

        if(show_confusion_matrix):
            confusion_matrix_dataframe = pd.DataFrame(confusion_matrix_metric.normalized(),
                                                      index = test_dataset.classes,
                                                      columns = test_dataset.classes)
            plt.figure(figsize = (16,12))
            ax = sn.heatmap(confusion_matrix_dataframe, annot=True, cmap = 'YlOrBr')
            ax.xaxis.tick_top()
            ax.set_xticks(range(1, len(test_dataset.classes)+1), test_dataset.classes, rotation=270, ha='right');

    else:
        precision = 0.0
        recall = 0.0
        f1score = 0.0


    # Como o Colab possui um tempo de execução muito variável,
    # o tempo pode ser definido como constante, usando um dos valores testados.
    average_time = 0.00565 # Resnet 50, 160x160

    if(test_time):
        model_input, _ = next(iter(test_dataloader))
        model_input = model_input.to(device)

        total_time = 0
        for i in range(time_test_iterations):
            t0 = datetime.now()
            outputs = model(model_input)
            t1 = datetime.now()
            total_time += (t1-t0).total_seconds()

        del model_input

        average_time = total_time/time_test_iterations

    final_score = 10*f1score*math.pow(1-average_time, 2)

    return final_score, {"average_time": average_time, "f1score": f1score, "precision": precision, "recall": recall}


In [ ]:
best_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50')
best_model.load_state_dict(torch.load(f'model_{timestamp}_best'))

test_results = test_model(model=best_model,
                          device=device,
                          test_dataset=test_dataset,
                          batch_size=1,
                          run_tests=True,
                          time_test_iterations=1000,
                          show_confusion_matrix=True)

print(f"Test data for model:")
print(f"  '-> Final Score: {test_results[0]}")
print(f"      |-> Average Time: {test_results[1]['average_time']}")
print(f"      '-> F1 Score:     {test_results[1]['f1score']}")
print(f"          |-> Precision: {test_results[1]['precision']}")
print(f"          '-> Recall:    {test_results[1]['recall']}")


Ordem Esperada para as classes no dataset de recicláveis:

- Class 0: bottle-blue
- Class 1: bottle-blue-full
- Class 2: bottle-blue5l
- Class 3: bottle-blue5l-full
- Class 4: bottle-dark
- Class 5: bottle-dark-full
- Class 6: bottle-green
- Class 7: bottle-green-full
- Class 8: bottle-milk
- Class 9: bottle-milk-full
- Class 10: bottle-multicolor
- Class 11: bottle-multicolorv-full
- Class 12: bottle-oil
- Class 13: bottle-oil-full
- Class 14: bottle-transp
- Class 15: bottle-transp-full
- Class 16: bottle-yogurt
- Class 17: glass-dark
- Class 18: glass-green
- Class 19: glass-transp
- Class 20: canister
- Class 21: cans
- Class 22: juice-cardboard
- Class 23: milk-cardboard
- Class 24: detergent-box
- Class 25: detergent-color
- Class 26: detergent-transparent
- Class 27: detergent-white